In [4]:
import os
import torch
from pathlib import Path
from ultralytics import YOLO

def convert_folder_to_torchscript(input_folder, output_folder):
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Tìm tất cả file .pt trong thư mục đầu vào
    pt_files = [f for f in os.listdir(input_folder) if f.endswith('.pt')]
    
    print(f"Tìm thấy {len(pt_files)} file .pt trong thư mục {input_folder}")
    
    for pt_file in pt_files:
        input_path = os.path.join(input_folder, pt_file)
        output_name = os.path.splitext(pt_file)[0] + '.torchscript.pt'
        output_path = os.path.join(output_folder, output_name)
        
        print(f"Đang xử lý: {pt_file}")
        
        try:
            # Cách 1: Sử dụng phương thức export tích hợp
            model = YOLO(input_path)
            model.export(format="torchscript", imgsz=640, simplify=True)
            
            # Tìm file xuất ra (thường trong thư mục runs/detect/train/weights/)
            export_path = Path("runs/export") / f"{os.path.splitext(pt_file)[0]}_torchscript.pt"
            if export_path.exists():
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                os.replace(str(export_path), output_path)
                print(f"Đã lưu mô hình TorchScript vào: {output_path}")
            else:
                print(f"Không tìm thấy file xuất: {export_path}")
                
        except Exception as e:
            print(f"Lỗi khi sử dụng export: {e}")
            
            try:
                # Cách 2: Sử dụng trace thay vì script
                print(f"Thử phương pháp trace cho {pt_file}...")
                model = YOLO(input_path)
                traced_model = model.model.eval().cpu()
                
                # Tạo input mẫu cho trace
                example_input = torch.rand(1, 3, 640, 640)
                
                # Áp dụng trace
                with torch.no_grad():
                    traced_model = torch.jit.trace(traced_model, example_input)
                    
                # Lưu mô hình
                traced_model.save(output_path)
                print(f"Đã lưu mô hình TorchScript (trace) vào: {output_path}")
            except Exception as e2:
                print(f"Lỗi khi sử dụng trace: {e2}")
    
    print("Chuyển đổi hoàn tất")

# Chỉ định thư mục đầu vào và đầu ra
input_folder = r"C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n"
output_folder = r"C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\models\torchscript"

# Thực hiện chuyển đổi
convert_folder_to_torchscript(input_folder, output_folder)

Tìm thấy 6 file .pt trong thư mục C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n
Đang xử lý: bamdinh.pt
Ultralytics 8.3.149  Python-3.10.16 torch-2.7.0+cpu CPU (Intel Core(TM) i7-14700K)
 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n\bamdinh.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.3 MB)

TorchScript: starting export with torch 2.7.0+cpu...
TorchScript: export success  1.7s, saved as 'C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n\bamdinh.torchscript' (10.5 MB)

Export complete (2.1s)
Results saved to C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n
Predict:         yolo predict task=detect model=C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n\ba

In [2]:
import os
from pathlib import Path
from ultralytics import YOLO

def convert_to_onnx(input_path, output_path=None, imgsz=640, opset=12):
    """
    Chuyển đổi mô hình YOLOv12 từ định dạng .pt sang ONNX
    
    Args:
        input_path: Đường dẫn đến file .pt
        output_path: Đường dẫn lưu file .onnx (tùy chọn)
        imgsz: Kích thước ảnh đầu vào
        opset: Phiên bản ONNX opset
    
    Returns:
        Path: Đường dẫn đến file ONNX đã được tạo
    """
    print(f"Đang chuyển đổi: {input_path}")
    
    # Tạo thư mục chứa output_path nếu được cung cấp
    if output_path:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    try:
        # Tải mô hình
        model = YOLO(input_path)
        
        # Xuất sang ONNX
        onnx_path = model.export(
            format="onnx",
            imgsz=imgsz,
            opset=opset,
            simplify=True,
            dynamic=True
        )
        
        # Di chuyển file nếu output_path được chỉ định
        if output_path and os.path.exists(onnx_path):
            os.replace(str(onnx_path), output_path)
            onnx_path = output_path
            
        print(f"Đã lưu mô hình ONNX vào: {onnx_path}")
        return onnx_path
        
    except Exception as e:
        print(f"Lỗi khi chuyển đổi mô hình sang ONNX: {e}")
        return None

if __name__ == "__main__":
    # Đường dẫn file .pt đầu vào (cần thay đổi theo môi trường Linux của bạn)
    input_path = "/root/Code/TestCycletimeMeiko/ModelYolov12n_PT/bamdinh.pt"
    
    # Đường dẫn lưu file ONNX đầu ra
    output_path = "/root/Code/TestCycletimeMeiko/ModelYolov12n_ONNX/bamdinh.onnx"
    
    # Thực hiện chuyển đổi
    convert_to_onnx(input_path, output_path)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/root/miniconda3/envs/ts_yolo12/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/root/miniconda3/envs/ts_yolo12/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/root/miniconda3/envs/ts_yolo12/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/root/miniconda3/envs/ts_yolo12/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launc

WARNING ⚠️ user config directory '/root/.config/Ultralytics' is not writeable, defaulting to '/tmp' or CWD.Alternatively you can define a YOLO_CONFIG_DIR environment variable for this path.
Đang chuyển đổi: /root/Code/TestCycletimeMeiko/ModelYolov12n_PT/bamdinh.pt
Ultralytics 8.3.150 🚀 Python-3.10.18 torch-2.2.1 CPU (Intel Core(TM) i7-14700K)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/root/Code/TestCycletimeMeiko/ModelYolov12n_PT/bamdinh.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.3 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.56...
ONNX: export success ✅ 4.4s, saved as '/root/Code/TestCycletimeMeiko/ModelYolov12n_PT/bamdinh.onnx' (10.0 MB)

Export complete (4.7s)
Results saved to /root/Code/TestCycletimeMeiko/ModelYolov12n_PT
Predict:         yolo predict task=detect model=/root/Code/TestCycletimeMeiko/ModelYolov12n_PT/bamdinh.onnx imgsz=640

In [1]:
!pip install onnx onnxruntime onnxruntime-gpu onnxslim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 3.1 MB/s eta 0:00:00a 0:00:03


In [4]:
import os
from pathlib import Path
from ultralytics import YOLO

def convert_to_onnx(input_path, output_path=None, imgsz=640, opset=12):
    """
    Chuyển đổi mô hình YOLOv12 từ định dạng .pt sang ONNX
    
    Args:
        input_path: Đường dẫn đến file .pt
        output_path: Đường dẫn lưu file .onnx (tùy chọn)
        imgsz: Kích thước ảnh đầu vào
        opset: Phiên bản ONNX opset
    
    Returns:
        Path: Đường dẫn đến file ONNX đã được tạo
    """
    print(f"Đang chuyển đổi: {input_path}")
    
    # Tạo thư mục chứa output_path nếu được cung cấp
    if output_path:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    try:
        # Tải mô hình
        model = YOLO(input_path)
        
        # Xuất sang ONNX
        onnx_path = model.export(
            format="onnx",
            imgsz=imgsz,
            opset=opset,
            simplify=True,
            dynamic=True
        )
        
        # Di chuyển file nếu output_path được chỉ định
        if output_path and os.path.exists(onnx_path):
            os.replace(str(onnx_path), output_path)
            onnx_path = output_path
            
        print(f"Đã lưu mô hình ONNX vào: {onnx_path}")
        return onnx_path
        
    except Exception as e:
        print(f"Lỗi khi chuyển đổi mô hình sang ONNX: {e}")
        return None

def convert_folder_to_onnx(input_folder, output_folder, imgsz=640, opset=12):
    """
    Chuyển đổi tất cả mô hình YOLOv12 từ định dạng .pt sang ONNX trong một thư mục
    
    Args:
        input_folder: Đường dẫn đến thư mục chứa các file .pt
        output_folder: Đường dẫn thư mục để lưu các file .onnx
        imgsz: Kích thước ảnh đầu vào
        opset: Phiên bản ONNX opset
    
    Returns:
        list: Danh sách các đường dẫn đến file ONNX đã được tạo
    """
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Tìm tất cả file .pt trong thư mục đầu vào
    pt_files = [f for f in os.listdir(input_folder) if f.endswith('.pt')]
    
    print(f"Tìm thấy {len(pt_files)} file .pt trong thư mục {input_folder}")
    
    converted_files = []
    for pt_file in pt_files:
        input_path = os.path.join(input_folder, pt_file)
        output_name = os.path.splitext(pt_file)[0] + '.onnx'
        output_path = os.path.join(output_folder, output_name)
        
        print(f"Đang xử lý: {pt_file}")
        
        # Thực hiện chuyển đổi
        result = convert_to_onnx(input_path, output_path, imgsz, opset)
        if result:
            converted_files.append(result)
    
    print(f"Chuyển đổi hoàn tất. Đã chuyển đổi {len(converted_files)}/{len(pt_files)} file")
    return converted_files

if __name__ == "__main__":
    # Đường dẫn thư mục chứa các file .pt đầu vào
    input_folder = "/root/Code/TestCycletimeMeiko/ModelYolov12n_PT"
    
    # Đường dẫn thư mục để lưu các file ONNX đầu ra
    output_folder = "/root/Code/TestCycletimeMeiko/ModelYolov12n_ONNX"
    
    # Thực hiện chuyển đổi cả thư mục
    convert_folder_to_onnx(input_folder, output_folder)

Tìm thấy 6 file .pt trong thư mục /root/Code/TestCycletimeMeiko/ModelYolov12n_PT
Đang xử lý: nganmach.pt
Đang chuyển đổi: /root/Code/TestCycletimeMeiko/ModelYolov12n_PT/nganmach.pt
Ultralytics 8.3.150 🚀 Python-3.10.18 torch-2.2.1 CPU (Intel Core(TM) i7-14700K)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/root/Code/TestCycletimeMeiko/ModelYolov12n_PT/nganmach.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.3 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.56...
ONNX: export success ✅ 3.9s, saved as '/root/Code/TestCycletimeMeiko/ModelYolov12n_PT/nganmach.onnx' (10.0 MB)

Export complete (4.2s)
Results saved to /root/Code/TestCycletimeMeiko/ModelYolov12n_PT
Predict:         yolo predict task=detect model=/root/Code/TestCycletimeMeiko/ModelYolov12n_PT/nganmach.onnx imgsz=640  
Validate:        yolo val task=detect model=/root/Code/TestCycletimeMeiko/Mode